##### 서울시 따릉이 이용정보 데이터

In [1]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


31. 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라

In [2]:
display(df['대여일자'].value_counts().to_frame().sort_index())
print(df['대여일자'].value_counts().index[0])

,count
대여일자,
2021-06-01,48081
2021-06-02,56100
2021-06-03,18985
2021-06-04,56788
2021-06-05,52730
2021-06-06,47787
2021-06-07,52623
2021-06-08,55623
2021-06-09,54507


2021-06-04


32. 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라

In [3]:
df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df['대여일자'].dt.day_name()
df[['day_name']].value_counts().to_frame()

,count
day_name,
Wednesday,110607
Friday,108877
Monday,107568
Tuesday,103704
Saturday,101299
Sunday,90918
Thursday,55977


33. 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라

In [4]:
ww = df.groupby(['day_name','대여소번호']).size().to_frame('size').sort_values(['day_name', 'size'], ascending=False).reset_index()
ww = ww.drop_duplicates('day_name', keep='first').reset_index(drop=True)
ww

,day_name,대여소번호,size
0,Wednesday,502,282
1,Tuesday,502,267
2,Thursday,2715,137
3,Sunday,502,372
4,Saturday,502,378
5,Monday,502,242
6,Friday,502,277


34. 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라. 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

In [5]:
qq = df[['연령대코드', '대여구분코드']].groupby(['연령대코드','대여구분코드']).size().unstack().fillna(0)
qq['전체횟수'] = qq['단체권'] + qq['일일권'] + qq['일일권(비회원)'] + qq['정기권']
qq['일일권/전체횟수'] = (qq['일일권'] + qq['일일권(비회원)']) / qq['전체횟수']
display(qq.sort_values('연령대코드', ascending=False)['일일권/전체횟수'])
qq.sort_values('연령대코드', ascending=False)['일일권/전체횟수'].index[0]

연령대코드
~10대    0.312525
70대~    0.048962
60대     0.042462
50대     0.065414
40대     0.110585
30대     0.166702
20대     0.239913
Name: 일일권/전체횟수, dtype: float64

'~10대'

In [6]:
d = df[df.대여구분코드.isin(['일일권', '일일권(비회원)'])].연령대코드.value_counts().sort_index()
t = df.연령대코드.value_counts().sort_index()
r = (d/t).sort_values(ascending=False)
print(r)
print('max ratio age:', r.index[0])

연령대코드
~10대    0.312525
20대     0.239913
30대     0.166702
40대     0.110585
50대     0.065414
70대~    0.048962
60대     0.042462
Name: count, dtype: float64
max ratio age: ~10대


35. 연령대별 평균 이동거리를 구하여라

In [7]:
# 지수 표현 없애기
pd.options.display.float_format = '{:.5f}'.format
# 원상복구
#pd.reset_option('display.float_format')

In [8]:
age = df[['연령대코드', '이동거리']].연령대코드.value_counts().sort_index().to_frame()
km = df[['연령대코드', '이동거리']].groupby('연령대코드').agg('sum')
ans = pd.concat([age, km], axis=1)
ans['평균 이동거리'] = ans['이동거리'] / ans['count']
ans['평균 이동거리']

연령대코드
20대    3211.89055
30대    3341.44386
40대    3514.85742
50대    3593.66810
60대    3538.14574
70대~   3085.03964
~10대   2634.42628
Name: 평균 이동거리, dtype: float64

In [9]:
df[['연령대코드', '이동거리']].groupby(['연령대코드']).agg('mean')

,이동거리
연령대코드,
20대,3211.89055
30대,3341.44386
40대,3514.85742
50대,3593.66810
60대,3538.14574
70대~,3085.03964
~10대,2634.42628


36. 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라

In [10]:
tt = df[df['연령대코드'] == '20대']


37. 6월 7일 ~10대의 “이용건수”의 중앙값은?

In [11]:
df.loc[(df.대여일자 == '2021-06-07') & (df.연령대코드 == '~10대')].이용건수.median()

1.0

38. 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라

In [12]:
wd = df[df.day_name.isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])]
wd[wd.대여시간.isin([6,7,8])][['대여시간', '대여소번호', '이용건수']].groupby(['대여시간', '대여소번호'])\
    .size().to_frame('이용횟수').sort_values(['대여시간', '이용횟수'], ascending=False).groupby('대여시간').head(3)

이용횟수
대여시간 대여소번호      
8    2701    119
     646     115
     1152     92
7    259     104
     230      77
     726      77
6    2744     45
     1125     40
     1028     36

39. 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라

In [13]:
df[df.이동거리 >= df.이동거리.mean()].이동거리.std()

5092.139707505356

40. 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라

In [14]:
df.성별 = df.성별.str.lower()
mm = df[df.성별 == 'm'].이동거리.mean()
fm = df[df.성별 == 'f'].이동거리.mean()
print(mm, fm)

3209.1108714530546 3468.5750252981957


In [15]:
df['sex'] = df['성별'].map(lambda x: '남' if x in ['M','m'] else '여')
df[['sex','이동거리']].groupby('sex').mean()

,이동거리
sex,
남,3209.11087
여,3468.57503
